In [17]:
#File for data cleaning
import pandas as pd

In [18]:
full_df = pd.read_csv("Wimbledon_featured_matches.csv")

In [2]:
variableList = ["_ace","_winner", "_double_fault","_break_pt_won","_break_pt_missed", "_unf_err" ]


In [19]:
test_df = full_df.loc[full_df["match_id"] == "2023-wimbledon-1301"]

In [20]:
test_df.head()

,match_id,player1,player2,elapsed_time,set_no,game_no,point_no,p1_sets,p2_sets,p1_games,...,p2_break_pt_won,p1_break_pt_missed,p2_break_pt_missed,p1_distance_run,p2_distance_run,rally_count,speed_mph,serve_width,serve_depth,return_depth
0,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:00:00,1,1,1,0,0,0,...,0,0,0,6.000,7.840,2,95.0,BC,NCTL,ND
1,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:00:38,1,1,2,0,0,0,...,0,0,0,5.253,7.094,1,118.0,B,CTL,ND
2,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:01:01,1,1,3,0,0,0,...,0,0,0,13.800,19.808,4,120.0,B,NCTL,D
3,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:01:31,1,1,4,0,0,0,...,0,0,0,51.108,75.631,13,130.0,BW,CTL,D
4,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:02:21,1,1,5,0,0,0,...,0,0,0,0.649,0.813,1,112.0,W,NCTL,NaN


In [25]:
# score_point_diff = Player_i_points - Player_notI_points
# serve_1_i_broken = server == i and p_i_Ace =! 1
# P_i_BrealPointToT = BreakPoint - BreakPoint miss



def get_row_momentum(row, i):
    momentum = 0
    not_i = 2 if i == 1 else 1
    score_point_diff = row[f"p{i}_points_won"] - \
        row[f"p{not_i}_points_won"]  # new row
    # new row
    serve_1_broken = 1 if row["server"] == not_i and row[f"p{not_i}_ace"] != 1 else 0
    # TODO: add momentum weights to all columns in the row
    
    
    for col in row:
        if(col == f"P{i}_ace" and col == 1):
            momentum = momentum + 2
        elif(col == f"P{i}_winner" and col == 1):
            momentum = momentum + 1
        elif(col == f"P{i}_double_fault" and col == 1):
            momentum = momentum - 2
        elif(col == f"P{i}_break_pt_won" and col == 1):
            momentum = momentum + 2
        elif(col == f"P{i}_break_pt_missed" and col == 1):
            momentum = momentum - 1
        elif(col == f"P{i}_unf_err" and col == 1):
            momentum = momentum - 2
        
        

    

    # TODO: add new vals in the cols

# TODO: add in double faults


def get_set_momentum(df, set_num, i):
    df_set = df.loc[::, df["set"] == set_num] #FIXXXXXXXXX
    for row in df_set:
        # TODO: change to create new or alter existing df
        # TODO: Winning long rallys
        # Create col for set and game momentum
        get_row_momentum(row, i)

# TODO: create game momentum

# TODO: Do something useful w it

In [27]:
test_row = test_df.iloc[11, :]

rally_df = test_df.loc[test_df["rally_count"] > 1]
rally_df = rally_df.loc[rally_df["server"] == 2]
rally_df = rally_df.loc[rally_df["serve_no"] == 1]

In [23]:
rally_df[["rally_count", "server"]]

,rally_count,server
11,4,2
12,4,2
13,3,2
17,3,2
24,3,2
...,...,...
267,2,2
274,4,2
276,3,2
288,9,2


In [24]:
i = 1
not_i = 2

# score_point_diff = test_row[f"p{i}_points_won"] - \
#     test_row[f"p{not_i}_points_won"]  # works

# serve_1_broken = 1 if test_row["server"] == not_i and test_row["rally_count"] > 1 else 0
# WORKS

In [4]:
print("P1" +  variableList[0])

P1_ace
